Github link: https://github.com/isabelleantaran/GSB-544-Projects/tree/main/lab

In [415]:
import pandas as pd
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

In [416]:
import numpy as np
from sys import exit
import plotnine as p9
import math

In [417]:
print(xmas.columns)

Index(['Day', 'Day.in.Words', 'Gift.Item', 'Verb', 'Adjective', 'Location'], dtype='object')


In [418]:
print(xmas)

    Day Day.in.Words  Gift.Item        Verb Adjective        Location
0     1        first  partridge         NaN       NaN  in a pear tree
1     2       second       dove         NaN    turtle             NaN
2     3        third        hen         NaN    french             NaN
3     4       fourth       bird         NaN   calling             NaN
4     5        fifth       ring         NaN    golden             NaN
5     6        sixth      goose    a-laying       NaN             NaN
6     7      seventh       swan  a-swimming       NaN             NaN
7     8       eighth       maid   a-milking       NaN             NaN
8     9        ninth       lady     dancing       NaN             NaN
9    10        tenth       lord   a-leaping       NaN             NaN
10   11     eleventh      piper      piping       NaN             NaN
11   12      twelfth    drummer    drumming       NaN             NaN


## Function 1: pluralize_gift()

In [419]:
print(xmas['Gift.Item'])

0     partridge
1          dove
2           hen
3          bird
4          ring
5         goose
6          swan
7          maid
8          lady
9          lord
10        piper
11      drummer
Name: Gift.Item, dtype: object


In [420]:
#| code-fold: false
def pluralize_gift(gift = None): 
    """
    Returns plural of a noun

    Parameters 
    ----------
    gift: str 
        A noun
        
    Return 
    ----------
    str
        Plural version
    """
    
    if hasattr(gift, '__iter__') and not isinstance(gift, str): # Check if gift is iterable but not a string 
        # Handles the case where the function is called on a pandas Series
        return type(gift)(pluralize_gift(g) for g in gift) #If gift is an iterable, call pluralize_gift() on each element g and return same type of collection, enabling vectorization

    if not isinstance(gift, str): # Check if gift is not a string
        exit("The input to pluralize_gift must be a string.") # Exit the program with an error message if input is not a string
    
    if 'oo' in gift: 
        gift = gift.replace("oo", "ee")
    elif gift[-1] == "y":
        gift= gift.replace("y", "ies")
    else:
        gift = gift + "s"
    
    return gift


In [421]:
# Unit Test

print(pluralize_gift("patridge"))
print(pluralize_gift("goose"))
print(pluralize_gift("lady"))

patridges
geese
ladies


In [422]:
pluralize_gift(xmas['Gift.Item'])

0     partridges
1          doves
2           hens
3          birds
4          rings
5          geese
6          swans
7          maids
8         ladies
9          lords
10        pipers
11      drummers
dtype: object

## Function 2: make_phrase()

In [423]:
xmas.dtypes

Day              int64
Day.in.Words    object
Gift.Item       object
Verb            object
Adjective       object
Location        object
dtype: object

In [424]:
#| code-fold: false

# Create dictionary outside the function
day_number_words = {
    1: "one", 2: "two", 3: "three", 4: "four", 
    5: "five", 6: "six", 7: "seven", 8: "eight", 
    9: "nine", 10: "ten", 11: "eleven", 12: "twelve"
}

# Create the Num.Word column using .map()
xmas["Num.Word"] = xmas["Day"].map(day_number_words)

In [425]:
def make_phrase(num, num_word, item, verb, adjective, location):
    """
    Returns phrase of the parameters.
    
    Parameters
    ----------
    num : int
        The day number 
    num_word : str
        The number word (e.g., "one", "two", "twelve")
    item : str
        The gift item within the phrase
    verb : str
        The action of the gift
    adjective : str
        The description of the gift
    location : str
        The location of gift
    
    Returns
    -------
    str
        The complete phrase for that day's gift
    """
    
    ## Step 1: Replace NAs with blank strings
    verb = verb.fillna("") if isinstance(verb, pd.Series) else ("" if pd.isna(verb) else str(verb))
    adjective = adjective.fillna("") if isinstance(adjective, pd.Series) else ("" if pd.isna(adjective) else str(adjective))
    location = location.fillna("") if isinstance(location, pd.Series) else ("" if pd.isna(location) else str(location))
    
    ## Step 2: If the day number is larger than 1, the gift items need pluralized!
    if num > 1:
        item_plural = pluralize_gift(item)
    else:
        item_plural = item
    
    ## Step 3: Figure out if a gift item starts with a vowel
    starts_with_vowel = item_plural[0].lower() in 'aeiou'
    
    ## Step 4: For the first day, if the gift item starts with a vowel, replace the day with "an", else "a"
    if num == 1:
        if starts_with_vowel:
            determiner = "an"
        else:
            determiner = "a"
    else:
        determiner = num_word
    
    ## Step 5: Put all of the pieces together into one string and return!
    parts = [determiner, adjective, item_plural, verb, location]
    # Filter out empty strings and join with spaces
    phrase = ' '.join(part for part in parts if part)
    
    return phrase

In [426]:
xmas.apply(lambda x: make_phrase(x['Day'], x['Num.Word'], x['Gift.Item'], x['Verb'], x['Adjective'], x['Location']), axis=1)

0     a partridge in a pear tree
1               two turtle doves
2              three french hens
3             four calling birds
4              five golden rings
5             six geese a-laying
6         seven swans a-swimming
7          eight maids a-milking
8            nine ladies dancing
9            ten lords a-leaping
10          eleven pipers piping
11      twelve drummers drumming
dtype: object

In [427]:
# Test
make_phrase(num = 10,
           num_word = "ten",
           item = "lord",
           verb = "a-leaping",
           adjective = "",
           location = "")

'ten lords a-leaping'

In [428]:
# Test
make_phrase(num = 12,
           num_word = "twelve",
           item = "drummer",
           verb = "drumming",
           adjective = "",
           location = "")

'twelve drummers drumming'

In [429]:
# Use the function to make a new column of the `xmas` column called `Full.Phrase` containing the sentences for the new gift on that day.
xmas['Full.Phrase'] = xmas.apply(lambda x: make_phrase(x['Day'], 
                                                        x['Num.Word'], 
                                                        x['Gift.Item'], 
                                                        x['Verb'], 
                                                        x['Adjective'], 
                                                        x['Location']), axis=1)

In [430]:
print(xmas["Full.Phrase"])

0     a partridge in a pear tree
1               two turtle doves
2              three french hens
3             four calling birds
4              five golden rings
5             six geese a-laying
6         seven swans a-swimming
7          eight maids a-milking
8            nine ladies dancing
9            ten lords a-leaping
10          eleven pipers piping
11      twelve drummers drumming
Name: Full.Phrase, dtype: object


In [431]:
# Verifies if the column called `Full.Phrase` has been added
print(xmas.columns) 

Index(['Day', 'Day.in.Words', 'Gift.Item', 'Verb', 'Adjective', 'Location',
       'Num.Word', 'Full.Phrase'],
      dtype='object')


In [432]:
# Ensures the `Full.Phrase` column accurately contains complete phrases for each day
print(xmas[['Day', 'Num.Word', 'Gift.Item', 'Full.Phrase']])

    Day Num.Word  Gift.Item                 Full.Phrase
0     1      one  partridge  a partridge in a pear tree
1     2      two       dove            two turtle doves
2     3    three        hen           three french hens
3     4     four       bird          four calling birds
4     5     five       ring           five golden rings
5     6      six      goose          six geese a-laying
6     7    seven       swan      seven swans a-swimming
7     8    eight       maid       eight maids a-milking
8     9     nine       lady         nine ladies dancing
9    10      ten       lord         ten lords a-leaping
10   11   eleven      piper        eleven pipers piping
11   12   twelve    drummer    twelve drummers drumming


## Function 3: sing_day()

In [433]:
def sing_day(dataset, num, phrase_col):
    """
    Returns the verse for a specific day of Christmas.
    
    Parameters
    ----------
    dataset : pandas.DataFrame
        Table containing at least a "Day" column and phrase_col with gift phrases.
        The data function that is applied to.
    num : int
        The day number 
    phrase_col : str
        Column name that holds the phrase text 
    
    Return
    -------
    str
        Entire section of song for a specific day

    """
    
    # Dictionary to map numbers to ordinal words
    num_word = {
        1: "first", 2: "second", 3: "third", 4: "fourth", 5: "fifth", 6: "sixth",
        7: "seventh", 8: "eighth", 9: "ninth", 10: "tenth", 11: "eleventh", 12: "twelfth"
    }
    
    # Check that the phrase column is in the dataset
    if phrase_col not in dataset.columns:
        exit(f"Column '{phrase_col}' not found in data.")
    
    # Step 1: Setup the intro line
    intro = "On the " + num_word[num] + " day of Christmas, my true love gave to me:"
    
    # Step 2: Sing the gift phrases
    gifts = ""
    for i in range(num, 0, -1):
        gift = dataset.loc[dataset["Day"] == i, phrase_col].values[0]
        
        if i == 1 and num != 1:
            # Last line (day 1) when there are other gifts
            # Add period to end, no comma before
            gifts += gift + "."
        elif i == 1:
            # Only gift (day 1 song) - add period
            gifts += gift + "."
        elif i == 2 and num != 1:
            # Second-to-last line - add " and" with no comma, then newline
            gifts += gift + " and\n"
        else:
            # All other middle lines - add comma and newline
            gifts += gift + ",\n"
    
    # Step 3: Put it all together and return
    return intro + "\n" + gifts

## Test your function

In [434]:
print(sing_day(xmas, 2, "Full.Phrase"))

On the second day of Christmas, my true love gave to me:
two turtle doves and
a partridge in a pear tree.


In [435]:
print(sing_day(xmas, 3, "Full.Phrase"))

On the third day of Christmas, my true love gave to me:
three french hens,
two turtle doves and
a partridge in a pear tree.


In [436]:
print(sing_day(xmas, 10, "Full.Phrase"))

On the tenth day of Christmas, my true love gave to me:
ten lords a-leaping,
nine ladies dancing,
eight maids a-milking,
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves and
a partridge in a pear tree.


In [437]:
print(sing_day(xmas, 12, "Full.Phrase"))

On the twelfth day of Christmas, my true love gave to me:
twelve drummers drumming,
eleven pipers piping,
ten lords a-leaping,
nine ladies dancing,
eight maids a-milking,
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves and
a partridge in a pear tree.


## Use Your Functions!

In [438]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")

In [439]:
print(xmas2.columns)

Index(['Day', 'Day.in.Words', 'Gift.Item', 'Verb', 'Adjective', 'Location'], dtype='object')


In [440]:
print(xmas2)

    Day Day.in.Words Gift.Item         Verb Adjective       Location
0     1        first     email          NaN       NaN  from Cal Poly
1     2       second     point          NaN      meal            NaN
2     3        third       pen          NaN      lost            NaN
3     4       fourth    review          NaN    course            NaN
4     5        fifth      exam          NaN  practice            NaN
5     6        sixth    grader      grading       NaN            NaN
6     7      seventh    senior    stressing       NaN            NaN
7     8       eighth       mom    a-calling       NaN            NaN
8     9        ninth     party      bumping       NaN            NaN
9    10        tenth      load   of laundry       NaN            NaN
10   11     eleventh    friend  goodbye-ing       NaN            NaN
11   12      twelfth      hour     sleeping       NaN            NaN


In [441]:
#| code-fold: false

# Create dictionary outside the function
day_number_words = {
    1: "one", 2: "two", 3: "three", 4: "four", 
    5: "five", 6: "six", 7: "seven", 8: "eight", 
    9: "nine", 10: "ten", 11: "eleven", 12: "twelve"
}

# Create the Num.Word column using .map()
xmas2["Num.Word"] = xmas2["Day"].map(day_number_words)

# Use the function to make a new column of the `xmas2` column called `Full.Phrase` containing the sentences for the new gift on that day.
xmas2['Full.Phrase'] = xmas2.apply(lambda x: make_phrase(x['Day'], 
                                                        x['Num.Word'], 
                                                        x['Gift.Item'], 
                                                        x['Verb'], 
                                                        x['Adjective'], 
                                                        x['Location']), axis=1)

In [442]:
print(xmas2["Full.Phrase"])

0         an email from Cal Poly
1                two meal points
2                three lost pens
3            four course reviews
4            five practice exams
5            six graders grading
6        seven seniors stressing
7           eight moms a-calling
8           nine parties bumping
9           ten loads of laundry
10    eleven friends goodbye-ing
11         twelve hours sleeping
Name: Full.Phrase, dtype: object


In [443]:
# Original 12 Days of Christmas song
print("The 12 Days of Christmas\n")
sing_full_song(xmas, "Full.Phrase")

The 12 Days of Christmas

On the first day of Christmas, my true love gave to me:
a partridge in a pear tree.

On the second day of Christmas, my true love gave to me:
two turtle doves and
a partridge in a pear tree.

On the third day of Christmas, my true love gave to me:
three french hens,
two turtle doves and
a partridge in a pear tree.

On the fourth day of Christmas, my true love gave to me:
four calling birds,
three french hens,
two turtle doves and
a partridge in a pear tree.

On the fifth day of Christmas, my true love gave to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves and
a partridge in a pear tree.

On the sixth day of Christmas, my true love gave to me:
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves and
a partridge in a pear tree.

On the seventh day of Christmas, my true love gave to me:
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two

In [444]:
# Christmas Version 2 - Cal Poly Surprise Song
print("The 12 Days of Christmas\n")
sing_full_song(xmas2, "Full.Phrase")

The 12 Days of Christmas

On the first day of Christmas, my true love gave to me:
an email from Cal Poly.

On the second day of Christmas, my true love gave to me:
two meal points and
an email from Cal Poly.

On the third day of Christmas, my true love gave to me:
three lost pens,
two meal points and
an email from Cal Poly.

On the fourth day of Christmas, my true love gave to me:
four course reviews,
three lost pens,
two meal points and
an email from Cal Poly.

On the fifth day of Christmas, my true love gave to me:
five practice exams,
four course reviews,
three lost pens,
two meal points and
an email from Cal Poly.

On the sixth day of Christmas, my true love gave to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points and
an email from Cal Poly.

On the seventh day of Christmas, my true love gave to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points and
an email 